<a href="https://colab.research.google.com/github/brendanpshea/intro_to_networks/blob/main/pcaps/PCAP_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Get ready for network analysis
!pip install scapy > /dev/null
!apt install tcpdump > /dev/null

## Detective Agency

In [4]:
from scapy.all import *
import random
from datetime import datetime, timedelta
import time
import ipaddress

def create_detective_agency_pcap():
    # Network devices with MAC addresses (fixed to use valid hex values)
    devices = {
        "router": {
            "ip": "192.168.1.1",
            "mac": "00:1A:2B:3C:4D:5E",
            "external_ip": "203.0.113.2",
            "external_mac": "00:1A:2B:3C:4D:5F"
        },
        "reception": {
            "ip": "192.168.1.10",
            "mac": "00:1B:2C:3D:4E:5F"
        },
        "detective1": {
            "ip": "192.168.1.11",
            "mac": "00:1C:2D:3E:4F:6A"
        },
        "detective2": {
            "ip": "192.168.1.12",
            "mac": "00:1D:2E:3F:4F:6B"
        },
        "research": {
            "ip": "192.168.1.13",
            "mac": "00:1E:2F:3F:4F:6C"
        },
        "scanner": {
            "ip": "192.168.1.14",
            "mac": "00:1F:2F:3F:4F:6D"
        },
        "security_camera": {
            "ip": "192.168.1.15",
            "mac": "00:2A:3B:4C:5D:6E"
        }
    }

    external_services = {
        "client_portal": "203.0.113.10",
        "email_server": "203.0.113.20",
        "cloud_storage": "203.0.113.30",
        "case_database": "203.0.113.40",
        "surveillance_feed": "203.0.113.50"
    }

    packets = []
    base_time = time.time()

    # ARP cache updates
    for device_name, device in devices.items():
        if device_name != "router":
            # ARP request
            arp_request = (
                Ether(src=device["mac"], dst="ff:ff:ff:ff:ff:ff")/
                ARP(
                    hwsrc=device["mac"],
                    psrc=device["ip"],
                    hwdst="00:00:00:00:00:00",
                    pdst=devices["router"]["ip"],
                    op=1  # who-has
                )
            )
            # ARP reply
            arp_reply = (
                Ether(src=devices["router"]["mac"], dst=device["mac"])/
                ARP(
                    hwsrc=devices["router"]["mac"],
                    psrc=devices["router"]["ip"],
                    hwdst=device["mac"],
                    pdst=device["ip"],
                    op=2  # is-at
                )
            )
            packets.extend([arp_request, arp_reply])

    # DHCP renewal
    for device_name, device in devices.items():
        if device_name != "router":
            # DHCP discover
            dhcp_discover = (
                Ether(src=device["mac"], dst="ff:ff:ff:ff:ff:ff")/
                IP(src="0.0.0.0", dst="255.255.255.255")/
                UDP(sport=68, dport=67)/
                BOOTP(
                    chaddr=device["mac"].replace(":", ""),
                    flags=0x8000
                )/
                DHCP(options=[("message-type", "discover"), "end"])
            )
            packets.append(dhcp_discover)

    # Regular DNS queries with full Ethernet frames
    for service in external_services:
        dns_query = (
            Ether(src=devices["reception"]["mac"], dst=devices["router"]["mac"])/
            IP(src=devices["reception"]["ip"], dst="8.8.8.8")/
            UDP(sport=random.randint(49152, 65535), dport=53)/
            DNS(
                rd=1,
                qd=DNSQR(qname=f"{service}.detective-agency.com")
            )
        )
        packets.append(dns_query)

    # HTTPS traffic with TCP handshake
    for detective in ["detective1", "detective2"]:
        src_port = random.randint(49152, 65535)

        # TCP 3-way handshake
        syn = (
            Ether(src=devices[detective]["mac"], dst=devices["router"]["mac"])/
            IP(src=devices[detective]["ip"], dst=external_services["client_portal"])/
            TCP(sport=src_port, dport=443, flags="S", seq=1000)
        )

        syn_ack = (
            Ether(src=devices["router"]["mac"], dst=devices[detective]["mac"])/
            IP(dst=devices[detective]["ip"], src=external_services["client_portal"])/
            TCP(sport=443, dport=src_port, flags="SA", seq=2000, ack=1001)
        )

        ack = (
            Ether(src=devices[detective]["mac"], dst=devices["router"]["mac"])/
            IP(src=devices[detective]["ip"], dst=external_services["client_portal"])/
            TCP(sport=src_port, dport=443, flags="A", seq=1001, ack=2001)
        )

        # Simulate TLS handshake with raw data
        tls_hello = (
            Ether(src=devices[detective]["mac"], dst=devices["router"]["mac"])/
            IP(src=devices[detective]["ip"], dst=external_services["client_portal"])/
            TCP(sport=src_port, dport=443, flags="PA", seq=1001, ack=2001)/
            Raw(load=b"\x16\x03\x03" + b"\x00"*50)  # Simplified TLS Client Hello
        )

        packets.extend([syn, syn_ack, ack, tls_hello])

    # NTP synchronization
    for device_name, device in devices.items():
        if device_name != "router":
            ntp_request = (
                Ether(src=device["mac"], dst=devices["router"]["mac"])/
                IP(src=device["ip"], dst="pool.ntp.org")/
                UDP(sport=random.randint(49152, 65535), dport=123)/
                Raw(load=b"\x1b" + b"\0"*47)  # NTP v3 request
            )
            packets.append(ntp_request)

    # ICMP echo requests (ping)
    for device_name, device in devices.items():
        if device_name != "router":
            ping = (
                Ether(src=device["mac"], dst=devices["router"]["mac"])/
                IP(src=device["ip"], dst="8.8.8.8")/
                ICMP(type=8, code=0)  # Echo request
            )
            ping_reply = (
                Ether(src=devices["router"]["mac"], dst=device["mac"])/
                IP(dst=device["ip"], src="8.8.8.8")/
                ICMP(type=0, code=0)  # Echo reply
            )
            packets.extend([ping, ping_reply])

    # HTTP traffic
    for detective in ["detective1", "detective2"]:
        http_get = (
            Ether(src=devices[detective]["mac"], dst=devices["router"]["mac"])/
            IP(src=devices[detective]["ip"], dst=external_services["client_portal"])/
            TCP(sport=random.randint(49152, 65535), dport=80)/
            Raw(load=b"GET / HTTP/1.1\r\nHost: detective-agency.com\r\n\r\n")
        )
        packets.append(http_get)

    # Add timestamps
    for i, packet in enumerate(packets):
        packet.time = base_time + (i * 0.1)  # Add 0.1 seconds between packets

    wrpcap("detective_agency.pcap", packets)
    print("Enhanced PCAP file 'detective_agency.pcap' has been created successfully.")

# Create the PCAP file
create_detective_agency_pcap()

Enhanced PCAP file 'detective_agency.pcap' has been created successfully.


In [7]:
%%bash
apt install tcpdump > /dev/null

In [14]:
%%bash
# See who's talking to who at the ethernet level
tcpdump -r detective_agency.pcap -e -n -c 10

14:50:15.957247 00:1b:2c:3d:4e:5f > ff:ff:ff:ff:ff:ff, ethertype ARP (0x0806), length 42: Request who-has 192.168.1.1 tell 192.168.1.10, length 28
14:50:16.057247 00:1a:2b:3c:4d:5e > 00:1b:2c:3d:4e:5f, ethertype ARP (0x0806), length 42: Reply 192.168.1.1 is-at 00:1a:2b:3c:4d:5e, length 28
14:50:16.157247 00:1c:2d:3e:4f:6a > ff:ff:ff:ff:ff:ff, ethertype ARP (0x0806), length 42: Request who-has 192.168.1.1 tell 192.168.1.11, length 28
14:50:16.257247 00:1a:2b:3c:4d:5e > 00:1c:2d:3e:4f:6a, ethertype ARP (0x0806), length 42: Reply 192.168.1.1 is-at 00:1a:2b:3c:4d:5e, length 28
14:50:16.357247 00:1d:2e:3f:4f:6b > ff:ff:ff:ff:ff:ff, ethertype ARP (0x0806), length 42: Request who-has 192.168.1.1 tell 192.168.1.12, length 28
14:50:16.457247 00:1a:2b:3c:4d:5e > 00:1d:2e:3f:4f:6b, ethertype ARP (0x0806), length 42: Reply 192.168.1.1 is-at 00:1a:2b:3c:4d:5e, length 28
14:50:16.557247 00:1e:2f:3f:4f:6c > ff:ff:ff:ff:ff:ff, ethertype ARP (0x0806), length 42: Request who-has 192.168.1.1 tell 192.168

reading from file detective_agency.pcap, link-type EN10MB (Ethernet), snapshot length 65535


In [15]:
%%bash
# Watch IP addresses communicate
tcpdump -r detective_agency.pcap -n 'ip' -c 10

14:50:17.157247 IP 0.0.0.0.68 > 255.255.255.255.67: BOOTP/DHCP, Request from 30:30:31:42:32:43, length 244
14:50:17.257247 IP 0.0.0.0.68 > 255.255.255.255.67: BOOTP/DHCP, Request from 30:30:31:43:32:44, length 244
14:50:17.357247 IP 0.0.0.0.68 > 255.255.255.255.67: BOOTP/DHCP, Request from 30:30:31:44:32:45, length 244
14:50:17.457247 IP 0.0.0.0.68 > 255.255.255.255.67: BOOTP/DHCP, Request from 30:30:31:45:32:46, length 244
14:50:17.557247 IP 0.0.0.0.68 > 255.255.255.255.67: BOOTP/DHCP, Request from 30:30:31:46:32:46, length 244
14:50:17.657247 IP 0.0.0.0.68 > 255.255.255.255.67: BOOTP/DHCP, Request from 30:30:32:41:33:42, length 244
14:50:17.757247 IP 192.168.1.10.51924 > 8.8.8.8.53: 0+ A? client_portal.detective-agency.com. (52)
14:50:17.857247 IP 192.168.1.10.60009 > 8.8.8.8.53: 0+ A? email_server.detective-agency.com. (51)
14:50:17.957247 IP 192.168.1.10.61904 > 8.8.8.8.53: 0+ A? cloud_storage.detective-agency.com. (52)
14:50:18.057247 IP 192.168.1.10.64110 > 8.8.8.8.53: 0+ A? case

reading from file detective_agency.pcap, link-type EN10MB (Ethernet), snapshot length 65535


In [17]:
%%bash
# See TCP connections being established
tcpdump -r detective_agency.pcap -n 'tcp[tcpflags] & (tcp-syn) != 0' -c 10

14:50:18.257247 IP 192.168.1.11.57071 > 203.0.113.10.443: Flags [S], seq 1000, win 8192, length 0
14:50:18.357247 IP 203.0.113.10.443 > 192.168.1.11.57071: Flags [S.], seq 2000, ack 1001, win 8192, length 0
14:50:18.657247 IP 192.168.1.12.60081 > 203.0.113.10.443: Flags [S], seq 1000, win 8192, length 0
14:50:18.757247 IP 203.0.113.10.443 > 192.168.1.12.60081: Flags [S.], seq 2000, ack 1001, win 8192, length 0
14:50:20.857247 IP 192.168.1.11.59109 > 203.0.113.10.80: Flags [S], seq 0:46, win 8192, length 46: HTTP: GET / HTTP/1.1
14:50:20.957247 IP 192.168.1.12.58450 > 203.0.113.10.80: Flags [S], seq 0:46, win 8192, length 46: HTTP: GET / HTTP/1.1


reading from file detective_agency.pcap, link-type EN10MB (Ethernet), snapshot length 65535


In [18]:
%%bash
# Look at DNS queries - detectives looking up websites
tcpdump -r detective_agency.pcap -n 'port 53' -c 5

14:50:17.757247 IP 192.168.1.10.51924 > 8.8.8.8.53: 0+ A? client_portal.detective-agency.com. (52)
14:50:17.857247 IP 192.168.1.10.60009 > 8.8.8.8.53: 0+ A? email_server.detective-agency.com. (51)
14:50:17.957247 IP 192.168.1.10.61904 > 8.8.8.8.53: 0+ A? cloud_storage.detective-agency.com. (52)
14:50:18.057247 IP 192.168.1.10.64110 > 8.8.8.8.53: 0+ A? case_database.detective-agency.com. (52)
14:50:18.157247 IP 192.168.1.10.63706 > 8.8.8.8.53: 0+ A? surveillance_feed.detective-agency.com. (56)


reading from file detective_agency.pcap, link-type EN10MB (Ethernet), snapshot length 65535


In [19]:
%%bash
# Show detailed Ethernet frames
tcpdump -r detective_agency.pcap -e -nn -vv -c 1

14:50:15.957247 00:1b:2c:3d:4e:5f > ff:ff:ff:ff:ff:ff, ethertype ARP (0x0806), length 42: Ethernet (len 6), IPv4 (len 4), Request who-has 192.168.1.1 tell 192.168.1.10, length 28


reading from file detective_agency.pcap, link-type EN10MB (Ethernet), snapshot length 65535


In [20]:
%%bash
# Show IP header details
tcpdump -r detective_agency.pcap -nn -vv 'ip' -c 1

14:50:17.157247 IP (tos 0x0, ttl 64, id 1, offset 0, flags [none], proto UDP (17), length 272)
    0.0.0.0.68 > 255.255.255.255.67: [udp sum ok] BOOTP/DHCP, Request from 30:30:31:42:32:43, length 244, Flags [Broadcast] (0x8000)
	  Client-Ethernet-Address 30:30:31:42:32:43
	  Vendor-rfc1048 Extensions
	    Magic Cookie 0x63825363
	    DHCP-Message (53), length 1: Discover


reading from file detective_agency.pcap, link-type EN10MB (Ethernet), snapshot length 65535


In [21]:
%%bash
# Show TCP header details
tcpdump -r detective_agency.pcap -nn -vvv 'tcp' -c 1

14:50:18.257247 IP (tos 0x0, ttl 64, id 1, offset 0, flags [none], proto TCP (6), length 40)
    192.168.1.11.57071 > 203.0.113.10.443: Flags [S], cksum 0xad91 (correct), seq 1000, win 8192, length 0


reading from file detective_agency.pcap, link-type EN10MB (Ethernet), snapshot length 65535


In [22]:
%%bash
# Show UDP header details
tcpdump -r detective_agency.pcap -nn -vv 'udp' -c 1

14:50:17.157247 IP (tos 0x0, ttl 64, id 1, offset 0, flags [none], proto UDP (17), length 272)
    0.0.0.0.68 > 255.255.255.255.67: [udp sum ok] BOOTP/DHCP, Request from 30:30:31:42:32:43, length 244, Flags [Broadcast] (0x8000)
	  Client-Ethernet-Address 30:30:31:42:32:43
	  Vendor-rfc1048 Extensions
	    Magic Cookie 0x63825363
	    DHCP-Message (53), length 1: Discover


reading from file detective_agency.pcap, link-type EN10MB (Ethernet), snapshot length 65535
